In [62]:
testVar = input("Input sentence without special characters and in all lowercase (IE !,.,?,$,@).:  \n")
testVar
testToxic = input("Input 0 if you think it's not toxic, 1 if you think what you said is toxic:  \n")

Input sentence without special characters and in all lowercase (IE !,.,?,$,@).:  
you sir are bad
Input 0 if you think it's not toxic, 1 if you think what you said is toxic:  
0


In [30]:
testToxic=int(testToxic)
type(testToxic)

int

In [31]:
from pyspark import SparkFiles
import pandas as pd
import hashlib
df = pd.read_csv("Resources/CleanedToxicTrainFinal.csv")
df.tail()

,id,comment_text,toxic
159566,ffe987279560d7ff,and for the second time of asking when your vi...,0
159567,ffea4adeee384e90,you should be ashamed of yourself \n\nthat is ...,0
159568,ffee36eab5c267c9,spitzer \n\numm theres no actual article for p...,0
159569,fff125370e4aaaf3,and it looks like it was actually you who put ...,0
159570,fff46fc426af1f9a,\nand i really dont think you understand i c...,0


In [32]:
df=df.append({'id':'test', 'comment_text':testVar,'toxic':testToxic}, ignore_index=True)
df.tail()

,id,comment_text,toxic
159567,ffea4adeee384e90,you should be ashamed of yourself \n\nthat is ...,0
159568,ffee36eab5c267c9,spitzer \n\numm theres no actual article for p...,0
159569,fff125370e4aaaf3,and it looks like it was actually you who put ...,0
159570,fff46fc426af1f9a,\nand i really dont think you understand i c...,0
159571,test,you sir are no good,0


In [33]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop = stopwords.words('english')
type(df)
df['comment_text'] = df['comment_text'].str.lower().str.split()
df['comment_text'] = df['comment_text'].apply(lambda x: [item for item in x if item not in stop])
df.tail()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/raymondchan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,id,comment_text,toxic
0,0000997932d777bf,"[explanation, edits, made, username, hardcore,...",0
1,000103f0d9cfb60f,"[daww, matches, background, colour, im, seemin...",0
2,000113f07ec002fd,"[hey, man, im, really, trying, edit, war, guy,...",0
3,0001b41b1c6bb37e,"[cant, make, real, suggestions, improvement, w...",0
4,0001d958c54c6e35,"[sir, hero, chance, remember, page, thats]",0


In [34]:
import hashlib
for index, row in df.iterrows():
#         print(index)
        sent = ''
        for item in row['comment_text']:
            sent +=item
#         print(sent)
        temp1 = hashlib.md5(sent.encode('utf-8'))
        temp2 = int(temp1.hexdigest(),16)
        df.loc[index,'hash'] = temp2

In [35]:
df.tail()

,id,comment_text,toxic,hash
159567,ffea4adeee384e90,"[ashamed, horrible, thing, put, talk, page, 12...",0,3.254371e+38
159568,ffee36eab5c267c9,"[spitzer, umm, theres, actual, article, prosti...",0,1.909200e+38
159569,fff125370e4aaaf3,"[looks, like, actually, put, speedy, first, ve...",0,1.004997e+38
159570,fff46fc426af1f9a,"[really, dont, think, understand, came, idea, ...",0,4.974771e+37
159571,test,"[sir, good]",0,1.085693e+38


In [36]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
model = LinearRegression()
model

# Assign X (data) and y (target)
X = df["hash"]
y = df["toxic"]
print(X.shape, y.shape)

(159572,) (159572,)


In [37]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [38]:
X=X.reshape(-1, 1)
y=y.reshape(-1, 1)
model.fit(X, y)
print(model)

/anaconda2/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.
/anaconda2/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)


In [39]:
print('Weight coefficients: ', model.coef_)
print('y-axis intercept: ', model.intercept_) 

Weight coefficients:  [[2.07000501e-43]]
y-axis intercept:  [0.09580862]


In [40]:
predictions = model.predict(X)
print(f"True output: {y[0]}")
print(f"Predicted output: {predictions[0]}")
print(f"Prediction Error: {predictions[0]-y[0]}")

True output: [0]
Predicted output: [0.09586937]
Prediction Error: [0.09586937]


In [41]:
from sklearn.metrics import mean_squared_error, r2_score
# Score the prediction with mse and r2
mse = mean_squared_error(y, predictions)
r2 = r2_score(y, predictions)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2 ): {r2}")

Mean Squared Error (MSE): 0.08665783218179327
R-squared (R2 ): 4.76454242726021e-09


In [42]:
data=np.column_stack([predictions,y,predictions-y]);
temp=pd.DataFrame(data,columns=["Predicted", "Actual", "Error"])


In [49]:
temp.tail()

,Predicted,Actual,Error
159567,0.095876,0.0,0.095876
159568,0.095848,0.0,0.095848
159569,0.095829,0.0,0.095829
159570,0.095819,0.0,0.095819
159571,0.095831,0.0,0.095831


In [51]:
output=temp.iloc[-1,:]

In [55]:
predictedOutput=output[0]
actualOutput=output[1]
errorOutput=output[2]

In [60]:
if(round(predictedOutput)==0):
    result='non-toxic'
else: 
    result='toxic'
result

'non-toxic'

In [61]:
round(predictedOutput)

0.0